## ReClor test의 question type 분류 시키기

In [2]:
import pandas as pd 
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from typing import List
fname = '../Data/ReClor/ReClor_test.jsonl'
data_dataset = load_dataset("json", data_files=fname, split="train")
data_loader = DataLoader(data_dataset, batch_size=1, shuffle=False)



/hdd/hjl8708/Anaconda3_envs/cut/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: Unable to find '/hdd/hjl8708/workspace/Inference/GPT_inference/../Data/ReClor/ReClor_test.jsonl'

In [32]:
def answers_to_string(answers: List) -> str:
    answer_str = ""
    for a, answer in enumerate(answers):
        while type(answer) == tuple:
            answer = answer[0]
        if a == len(answers) - 1:
            answer_str += f"{a+1}: {answer}"
        else:
            answer_str += f"{a+1}: {answer}, "
    return answer_str

def template_to_string(template: str, context: str, question: str, options: List[str]) -> str:
    # option combinate
    option_str = answers_to_string(options)
    return template.format(context=context, question=question, options=option_str)

template = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction: You are given a logical reasoning question and predefined types of questions. Each type has its unique type code such as 'N1'. write the code of the question type of following question.
There are predefined 17 types of questions. The number of the question type and the description of the question type are as follows: 
N1. "Necessary Assumptions": identify the claim that must be true or is required in order for the argument to work.
N2. "Sufficient Assumptions": identify a sufficient assumption, that is, an assumption that, if added to the argument, would make it logically valid.
N3. "Strengthen": identify information that would strengthen an argument
N4. "Weaken": identify information that would weaken an argument
N5. "Evaluation": identify information that would be useful to know to evaluate an argument
N6. "Implication": identify something that follows logically from a set of premises
N7. "Conclusion/Main Point": identify the conclusion/main point of a line of reasoning
N8. "Most Strongly Supported": find the choice that is most strongly supported by a stimulus
N9. "Explain or Resolve": identify information that would explain or resolve a situation
N10. "Principle": identify the principle, or find a situation that conforms to a principle, or match the principles
N11. "Dispute": identify or infer an issue in dispute
N12. "Technique": identify the technique used in the reasoning of an argument
N13. "Role": describe the individual role that a statement is playing in a larger argument
N14. "Identify a Flaw": identify a flaw in an argument’s reasoning
N15. "Match Flaws": find a choice containing an argument that exhibits the same flaws as the passage’s argument
N16. "Match Structures": match the structure of an argument in a choice to the structure of the argument in the passage
N17. "Others": other types of questions which are not included by the above

Following logical reasoning question.
Context: {context}
Question: {question}
Options: {options}
The code of the question type of this question is N
""".strip()

for ix, x in enumerate(data_loader):
    context = x["context"][0] # str
    question = x["question"][0]
    answers = x["answers"] # List 
    answers = [x[0] for x in answers]
    question_type = x["question_type"][0] # int
    id_string = x["id_string"][0] # str
    
    input_string = template_to_string(template, context, question, answers)
    
    

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

Instruction: You are given a logical reasoning question and predefined types of questions. Each type has its unique type code such as 'N1'. write the code of the question type of following question.
There are predefined 17 types of questions. The number of the question type and the description of the question type are as follows: 
N1. "Necessary Assumptions": identify the claim that must be true or is required in order for the argument to work.
N2. "Sufficient Assumptions": identify a sufficient assumption, that is, an assumption that, if added to the argument, would make it logically valid.
N3. "Strengthen": identify information that would strengthen an argument
N4. "Weaken": identify information that would weaken an argument
N5. "Evaluation": identify information that would be useful to know to evaluate an argument
N6. "Implica

In [9]:
# path
import pandas as pd 
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from typing import List
filepath = 'qtype_result/ReClor_test_result.jsonl'

data = pd.read_json(filepath, lines=True)

resp_exs = [data.response.values[x] for x in [0,2,5]]
resp_exs

def response_to_code(response: str):
    # regex를 통해서 'N\d'를 찾아서 숫자를 반환
    import re
    regex = r'N\d'
    match = re.search(regex, response)
    if match:
        return int(match.group()[1:])
    else:
        # 문자열에서 나타나는 첫 번째 숫자를 찾는다
        return extract_first_num_in_string(response)
    
def extract_first_num_in_string(string: str):
    result = ''.join([x for x in string if x.isdigit()])
    if result == '':
        return -1
    else: 
        return int(''.join([x for x in string if x.isdigit()]))-1


In [14]:
print(f'''response: {resp_exs[0]}\npredicted num: {response_to_code(resp_exs[0])}\nlabel: {data.label.question_type[0]}''')

AttributeError: 'DataFrame' object has no attribute 'label'